In [2]:
#colab 용 konlpy 모듈
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-04-24 07:31:57--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601a:18::a27d:712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-04-24 07:31:57--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc42ed3f800ba2a10d964a58f17e.dl.dropboxusercontent.com/cd/0/get/B6yppIwxvV1S3qZqBnWUD8vMJteQmKHQyqJPKpO6VNrdVlSSdTbTuOHRSi6do-VOWENkCraxzNZNtMsEuBsS1xus0u2HeqsRKF6_XahfBeazJp4paZELELaoBRteF7U4ZX11UmzVE6lTYOVleg8zNvVgrOIqMD7QtTob1S5MweU3kZFK22EVXAGjl9ZQV2txANo/file?dl=1# [following]
--2023-04-24 07:31:58--  https://uc42ed3f800ba2a10d964a58f17e.dl.dropboxusercontent.com/cd/0/get/B6yppIwxvV1S

In [5]:
# 기존 word2vec 로 학습한 모델을 doc2vec 기반의 모델로 변경후 d2v 이라는 이름으로 저장
import ssl
import certifi
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt
from tqdm import tqdm

train_data = pd.read_csv('/content/sample_data/news.csv')
print(train_data)
train_data['document'] = train_data['title']+' '+train_data['contents']
train_data['label'] = train_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(len(train_data))

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
okt = Okt()

# Doc2Vec 학습을 위해 TaggedDocument 생성
tagged_data = [TaggedDocument(words=okt.morphs(_d, stem=True), tags=[str(i)]) for i, _d in enumerate(tqdm(train_data['document']))]

model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=6, dm=1)

model.save('review_d2v.model')

       idx                                           title  \
0        1                 보라 데보라 유인나윤현민 로코 장인들의 연애 루저 탈출기   
1        2                          홍김동전 주우재홍진경 극과 극 연애 조언   
2        3                    유퀴즈환승연애 제작진 뭉쳐 대 연애예능 소소연 론칭   
3        4             한혜진 살 차이상사와 비밀 연애 악조건 다 갖고 있다 연애의참견   
4        5                       환승연애  대 리얼리티 제작소년 소녀 연애하다   
...    ...                                             ...   
7446  7447                   서울 코리아오픈 국제휠체어테니스 대회 일간 열전 마쳐   
7447  7448                    한은 국제유가 상방압력 우세  수요회복 수준이 관건   
7448  7449                                  국제 설탕값 년 만에 최고   
7449  7450                     동대구역 광장 공공디자인 국제 디자인 어워즈 수상   
7450  7451  유류세 조정 찬물 끼얹은 국제유가 세수물가 동시타격 세수부족유가상승 고심 깊은 정부   

                                               contents keyword  
0     마이데일리  노한빈 기자  새 오리지널 드라마 보라 데보라연출 이태곤 서민정 극본 ...      연애  
1      예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명...      연애  
2     서울뉴스 윤효정 기자  대들의 연애 리얼리티 소소연이 론칭된다 엔터테인먼트 스트리밍...  

100%|██████████| 7451/7451 [11:03<00:00, 11.23it/s]


In [93]:
import ssl
import certifi
import pandas as pd
import urllib.request
from gensim.models.doc2vec import Doc2Vec
from konlpy.tag import Okt
from tqdm import tqdm

# 데이터 로드
news_data = pd.read_csv('/content/sample_data/news.csv')

# 데이터 전처리
news_data['document'] = news_data['title'] + ' ' + news_data['contents']
news_data['label'] = news_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
news_data = news_data.dropna(how='any')

# 불용어(stopwords)와 형태소 분석기(okt) 준비
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
okt = Okt()

# Doc2Vec 모델 로드
model = Doc2Vec.load('review_d2v.model')

# 추천 할 문서 갯수 선정 
num_recommendations = 5

# 추천할 문서 선택 (데이터프레임의 index 기준으로)
consumed_doc_idx = 1

# 선택한 문서 출력
print(news_data['title'].iloc[consumed_doc_idx])
print(news_data['contents'].iloc[consumed_doc_idx])

# 선택한 문서를 기반으로 유사한 문서들 추천
similar_docs = model.dv.most_similar(consumed_doc_idx, topn=num_recommendations)

print(similar_docs)



홍김동전 주우재홍진경 극과 극 연애 조언
 예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명을 체인지하는 피땀눈물의 구개념 버라이어티 홍김동전은 국내  플랫폼 웨이브에서 지난해 전체 프로그램 위를 시작으로 위를 거쳐 최근 위를 기록한 데 이어 웨이브 신규유료가입 견인 콘텐츠 예능 부문 위를 차지했다 또한 굿데이터코퍼레이션에 따르면 홍김동전은 월 주 차  통합 화제성 비드라마 부문 위 홍김동전에 출연한 방탄소년단 지민은  통합 화제성 비드라마 출연자 부문 위에 오르며 연일 화제가 되고 있다오는 일 방송되는 홍김동전 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄으로 꾸며진다 고등학교 학창 시절로 돌아간 멤버들은 부산 창원 등 각자의 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 여 고교 후배의 썸 갈등 사연에 자신의 산전수전 경험을 토대로 조언을 한다홍김동전 멤버들이 고교 후배들의 생생한 연애의 참견에 나선다 이 중 년 동안 애매하게 연락을 이어가는 여고생의 썸 사연이 등장하자 연애도 남사친도 아닌 애매한 썸 관계에 홍김동전 경력직 선배들의 조언이 무수히 쏟아졌다는 전언이다특히 주우재는 연애 조언 경력자답게 사연을 듣자마자 제대로 된 남자는 좋아하는 여자를 헷갈리게 하지 않는다며 프로 참견러의 면모를 발산해 시선을 사로잡는다 주우재는 장담하는데 그 남자는 여러 여자에게 같은 톡을 보냈을 가능성이 크다 절대 만나지 마라고 매운맛 팩폭을 이어간다 과거 첫사랑 때문에 일주일 동안 밥을 먹지 않았던 주우재인 만큼 경험에서 우러나오는 조언이 후배의 정신을 혼미하게 만들었다는 후문 김숙 역시 주우재의 말에  공감하며 후배들을 위해 조언을 아끼지 않는다고 해 주우재가 프로 연애 참견러로서 어떤 활약을 펼쳤을지 궁금증이 모인다하지만 그런 주우재에게 반기를 든 멤버가 있었으니 바로 홍진경 홍진경은 썸남이 있다는 자체가 소중한 일이라며 주우재와 도 다른 연애 참견으로 후배들의 공감을 유발한다 특히 홍진경은 지금은 모른다

In [94]:
for doc_idx, sim in similar_docs:
        row = news_data.iloc[int(doc_idx)]
        doc_title = row['title']
        doc_keyword = row['keyword']
        doc_content = row['contents']
        print(f"유사도: {sim}, 제목: {doc_title}, 카테고리: {doc_keyword}, 본문: {doc_content}")

유사도: 0.6261309385299683, 제목: 트와이스빅뱅으로 케이팝에 눈 뜬 데이비드 용 잠재력 느꼈다 인터뷰, 카테고리: 문화, 본문: 김채연 기자 다국적기업의 이자 변호사로 활동하는 데이비드 용이 한국 활동을 시작하게 된 계기를 밝혔다최근 데이비드 용은 서울 광진구 자양동에 위치한 한 카페에서 을 만나 신곡 아마도 우린 발매 인터뷰를 진행했다데이비드용은 세계적으로 주목받는 다국적 기업의 사업가이자 변호사로 이번 신곡 아마도 우린은 약 개월 만에 컴백하는 것이다 아티스트 활동을 통해  업계에 선한 영향력을 끼칠 수 있는 인플루언서 를 꿈꾸는데이비드용은 아티스트로서도 꾸준히 활약하며 진정한 멀티 플레이어의 면모를 보여주고 있다이날 데이비드 용은 자신의 음악활동과 관련해 년 전 엔터테인먼트 사업차 한국에 왔을 때 윤민수 와 인연을 맺게 됐다 사업에만 몰두하기보다 직접 가수로 활동해보면 비즈니스 적으로 저에게 더 도움이 될 것 같았다 사실 대학 입학 전에 대만의 한 엔터테인먼트에서 연예계 활동과 관련해 제안이 온 적이 있었다 그때는 공부가 더 중요해서 거절했는데 뒤늦게 다시 도전한다는 의의가 있다며 년 동안 사업도 변호사도 지금 하고 있는 일까지 해야하는 일이 많았는데 지금은 잠시 떨어져서 내가 원하는 일을 좀 더 하고 있는 것 같다고 이야기했다다국적 기업을 운영하던 기업가이자 변호사인 그가 어떻게 한국에 와 가수로 활동하게 됐을까데이비드 용은 동남아시아에서 의 인기가 상당하다 케이팝은 전세계에서 유명하지만 동남아시아에서 특히 큰 사랑을 받고 있다 제가 케이팝에 대해 생소했을 때 제 친동생이 트와이스를 너무 좋아해서 콘서트에 갔다 살 친구들이 울고 감동받는 모습에 큰 충격을 받았다 케이팝의 잠재력을 느끼고 그때부터 주의깊게 살펴보게 됐다고 설명했다당시 인상깊었던 케이팝 아티스트가 있었냐고 묻자 데이비드 용은 그 뒤로 케이팝 콘서트를 자주 다녔다 빅뱅 선배님들이 멋있어서 더 관심을 갖게 되고 케이팝의 매력을 느꼈다 또 아이유 선배님의 연기 노래 등 올라운더의